In [1]:
import pdfminer

In [2]:
from pdfminer.high_level import extract_text
 
msa1 = extract_text("data/msa1.pdf")
msa2 = extract_text("data/msa2.pdf")
msa3 = extract_text("data/msa3.pdf")
msa4 = extract_text("data/msa4.pdf")

ssa1 = extract_text("data/ssa1.pdf")
ssa2 = extract_text("data/ssa2.pdf")
ssa3 = extract_text("data/ssa3.pdf")
ssa4 = extract_text("data/ssa4.pdf")

In [3]:
from transformers import pipeline, BertTokenizer, BertModel, BertForNextSentencePrediction, BertConfig
import torch

In [5]:
import re 

suffixlist = ['INC', 'L.P', 'L. P.', 'LLP', 'LLLP', 'Corp']

suffixes = '\.?|'.join([suffix for suffix in suffixlist])
print(suffixes)

INC\.?|L.P\.?|L. P.\.?|LLP\.?|LLLP\.?|Corp


In [6]:
suffixes

'INC\\.?|L.P\\.?|L. P.\\.?|LLP\\.?|LLLP\\.?|Corp'

In [73]:
import re 

reg_pattern1 = "(?!Master Service Agreement|.+MASTER SERVICE AGREEMENT -)(^.+(Incorporated|L\.?\s?P\.?|INC\.?|LLP\.?|LLLP\.?|\bCorp\.?\b))"

# reg_pattern2 = '''(Master Service Agreement\s?)(-\s)(.+(Incorporated|L\.?\s?P\.?|INC\.?|LLP\.?|LLLP\.?|\bCorp\.?\b))'''

pattern1 = re.compile(reg_pattern1, re.M)
# pattern2 = re.compile(reg_pattern2)
pattern1

re.compile(r'(?!Master Service Agreement|.+MASTER SERVICE AGREEMENT -)(^.+(Incorporated|L\.?\s?P\.?|INC\.?|LLP\.?|LLLP\.?|\x08Corp\.?\x08))',
re.MULTILINE|re.UNICODE)

In [74]:
matches = pattern1.findall(msa1[0:1000])

In [76]:
for match in matches:
    print(match)

('BRIGHTSOURCE CONSTRUCTION MANAGEMENT, INC.', 'INC.')
('CHEVRON TECHNOLOGY VENTURES, A DIVISION of CHEVRON U.S.A. INC.', 'INC.')


In [77]:
matches2 = pattern1.findall(msa2[0:1000])

In [83]:
for match in matches2:
    print(match)

('ICON Clinical Research, L. P.', 'L. P.')
('Corcept Therapeutics Incorporated', 'Incorporated')


In [84]:
matches3 = pattern1.findall(msa3[0:1000])

In [85]:
for match in matches3:
    print(match)

In [86]:
matches4 = pattern1.findall(msa4[0:1000])

In [87]:
for match in matches4:
    print(match)

In [88]:
print(msa3[0:1000])

Master Service Agreement

EX-10.21 3 dex1021.htm MASTER SERVICE AGREEMENT

MASTER SERVICES AGREEMENT

Exhibit 10.21

THIS MASTER SERVICES AGREEMENT (the “MSA”), effective as of January 01, 2005 (the “Effective Date”), is

made and entered into by and between ProQuest Company, a Delaware corporation having its principal offices at 300 North
Zeeb Road, Ann Arbor, MI 48106 (“Client Parent”), and International Business Machines Corporation, a New York
corporation having its principal offices at Route 100, Somers, New York 10589 (“Supplier Parent”).
 

RECITALS

WHEREAS, Client Parent and Supplier Parent have engaged in extensive negotiations, discussions and due diligence

that have culminated in the formation of the contractual relationship described in this MSA; and
 

WHEREAS, Client Parent desires to procure from Supplier Parent, and Supplier Parent desires to provide to Client
Parent, the information technology products and services described in this MSA and in Service Agreements ente

In [147]:
reg_pattern2 = r"(entered into by and between\s|\band\b\s|between\s)(.+(Incorporated|L\.?\s?P\.?|Inc\.|INC\.?|LLP\.?|LLLP\.?|\bCompany\b|Corporation|\bCorp\.?\b))"
pattern2 = re.compile(reg_pattern2, re.M)

In [148]:
matches3 = pattern2.findall(msa3[0:1000], re.M)

In [149]:
for match in matches3:
    print(match)

('and ', 'entered into by and between ProQuest Company', 'Company')
('and ', 'International Business Machines Corporation', 'Corporation')


In [143]:
print(msa3[0:1000])

Master Service Agreement

EX-10.21 3 dex1021.htm MASTER SERVICE AGREEMENT

MASTER SERVICES AGREEMENT

Exhibit 10.21

THIS MASTER SERVICES AGREEMENT (the “MSA”), effective as of January 01, 2005 (the “Effective Date”), is

made and entered into by and between ProQuest Company, a Delaware corporation having its principal offices at 300 North
Zeeb Road, Ann Arbor, MI 48106 (“Client Parent”), and International Business Machines Corporation, a New York
corporation having its principal offices at Route 100, Somers, New York 10589 (“Supplier Parent”).
 

RECITALS

WHEREAS, Client Parent and Supplier Parent have engaged in extensive negotiations, discussions and due diligence

that have culminated in the formation of the contractual relationship described in this MSA; and
 

WHEREAS, Client Parent desires to procure from Supplier Parent, and Supplier Parent desires to provide to Client
Parent, the information technology products and services described in this MSA and in Service Agreements ente

In [151]:
import nltk
from nltk.corpus import stopwords

nltk.download('punkt')
stop_words = set(stopwords.words('english')) 

sentence = 'entered into by and between'

# from: https://scikit-learn.org/stable/modules/feature_extraction.html

from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer

class LemmaTokenizer:
    """
    Interface to the WordNet lemmatizer from nltk
    """
    ignore_tokens = [',', '.', ';', ':', '"', '``', "''", '`']
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t) for t in word_tokenize(doc) if t not in self.ignore_tokens]
    
# Demonstrate the job of the tokenizer
nltk.download('wordnet')
tokenizer=LemmaTokenizer()

tokenizer(sentence)

[nltk_data] Downloading package punkt to /Users/jama/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jama/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['entered', 'into', 'by', 'and', 'between']

In [152]:
from nltk.stem import PorterStemmer

In [154]:
from nltk.tokenize import word_tokenize
ps = PorterStemmer()

words = word_tokenize(sentence)

for word in words:
    print(word + ":" + ps.stem(word))
    if word in stop_words:
        print("word : ", word, "TRUE")

entered:enter
into:into
word :  into TRUE
by:by
word :  by TRUE
and:and
word :  and TRUE
between:between
word :  between TRUE
